<a href="https://colab.research.google.com/github/steelpipe75/kagglebook-for-colab/blob/master/ch03/ch03-03-multi_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
import importlib
import sys
import subprocess

# Google Colab 上で実行しているかどうかを判断するフラグ
ON_COLAB = "google.colab" in sys.modules
print(f"ON_COLAB: {ON_COLAB}")

if ON_COLAB:
    USE_GIT = True # Gitを使う
    # USE_GIT = False # Gitを使わない

    print(f"USE_GIT: {USE_GIT}")
    if USE_GIT:
        !git clone https://github.com/ghmagazine/kagglebook.git
    else:
        # Google Drive にマウントする
        drive = importlib.import_module("google.colab.drive")
        drive.mount("/content/drive/")

        import os
        colab_dir = "/content/drive/MyDrive/kagglebook/" # データ置き場

ON_COLAB: True
USE_GIT: True
Cloning into 'kagglebook'...
remote: Enumerating objects: 95, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (73/73), done.
remote: Total 95 (delta 19), reused 87 (delta 18), pack-reused 1 (from 1)
Receiving objects: 100% (95/95), 3.13 MiB | 9.97 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [3]:
# -----------------------------------
# データの結合
# -----------------------------------
# データの読み込み
if ON_COLAB:
    if USE_GIT:
        train = pd.read_csv('/content/kagglebook/input/ch03/multi_table_train.csv')
        product_master = pd.read_csv('/content/kagglebook/input/ch03/multi_table_product.csv')
        user_log = pd.read_csv('/content/kagglebook/input/ch03/multi_table_log.csv')
    else:
        train = pd.read_csv(os.path.join(colab_dir, 'input/ch03/multi_table_train.csv'))
        product_master = pd.read_csv(os.path.join(colab_dir, 'input/ch03/multi_table_product.csv'))
        user_log = pd.read_csv(os.path.join(colab_dir, 'input/ch03/multi_table_log.csv'))
else:
    train = pd.read_csv('../input/ch03/multi_table_train.csv')
    product_master = pd.read_csv('../input/ch03/multi_table_product.csv')
    user_log = pd.read_csv('../input/ch03/multi_table_log.csv')

-----------------------------------
図の形式のデータフレームがあるとする
train         : 学習データ（ユーザID, 商品ID, 目的変数などの列がある）
product_master: 商品マスタ（商品IDと商品の情報を表す列がある）
user_log      : ユーザの行動のログデータ（ユーザIDと各行動の情報を表す列がある）

In [4]:
# 商品マスタを学習データと結合する
train = train.merge(product_master, on='product_id', how='left')

In [5]:
# ログデータのユーザごとの行数を集計し、学習データと結合する
user_log_agg = user_log.groupby('user_id').size().reset_index().rename(columns={0: 'user_count'})
train = train.merge(user_log_agg, on='user_id', how='left')